In [41]:
# Importar módulo necesario
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, udf, regexp_replace, floor


In [33]:
# Crear una instancia de SparkSession
spark = SparkSession.builder.getOrCreate()

In [34]:
# Leer el archivo CSV
df = spark.read.csv("./santander/titofin_inventario-20240327.csv", sep=";", header=True)

In [35]:
# Mostrar esquema del DataFrame
df.printSchema()

root
 |-- cartera: string (nullable = true)
 |-- especie_codigo_sam: string (nullable = true)
 |-- secid: string (nullable = true)
 |-- rubro_st: string (nullable = true)
 |-- valor_nominal: string (nullable = true)
 |-- valor_contable: string (nullable = true)
 |-- fecha_incorporacion_tenencia: string (nullable = true)
 |-- fecha_vencimiento: string (nullable = true)
 |-- tipo_valoracion: string (nullable = true)
 |-- isin: string (nullable = true)



In [36]:
# pasar fecha en formato dd/mm/yyyy a formato yyyy-mm-dd
# Definir la función format_date como un UDF
# Define the format_date function as a UDF
def format_date(date):
    if date is None:
        return None
    return date[6:] + "-" + date[3:5] + "-" + date[:2]

format_date_udf = udf(format_date)

In [37]:
df_cartera_de_especies_2 = df.withColumn("secid", when(col("secid").isNull() | (col("secid") == "NI"), None).otherwise(col("secid"))) \
    .withColumn("isin", when((col("isin").isNull()) | (col("isin") == "NI"), None).otherwise(col("isin"))) \
    .withColumn("rubro_altair", when(col("rubro_st").isNull(), None).otherwise(regexp_replace(col("rubro_st"), '-', ''))) \
    .withColumn("fecha_incorporacion_tenencia", format_date_udf(col("fecha_incorporacion_tenencia"))) \
    .withColumn("fecha_vencimiento", format_date_udf(col("fecha_vencimiento"))) \
    .withColumn("especie", col("especie_codigo_sam")).drop("especie_codigo_sam", "rubro_st") \
    .withColumn("valor_nominal", floor(col("valor_nominal")))
    # Sacarle los decimales a la columna "valor_nominal"

df_cartera_de_especies_2.show(5)
                            
                            

+-------+-----+-------------+--------------+----------------------------+-----------------+---------------+----+------------+-------+
|cartera|secid|valor_nominal|valor_contable|fecha_incorporacion_tenencia|fecha_vencimiento|tipo_valoracion|isin|rubro_altair|especie|
+-------+-----+-------------+--------------+----------------------------+-----------------+---------------+----+------------+-------+
|    TRD| null|         8118|     4241655.0|                  2020-09-08|       2038-01-09|              1|null|     1250405| ARN38D|
|    TRD| null|       151942|   58289663.51|                  2020-09-08|       2041-07-09|              2|null|     1250405| ARN41E|
|    TRD| null|        77966|   45464302.83|                  2020-09-08|       2030-07-09|              2|null|     1250405| AR30DR|
|    TRD| null|         2191|    1160488.27|                  2020-09-08|       2029-07-09|              2|null|     1250405| AR30D5|
|    TRD| null|        50000|    27434250.2|                  

In [38]:
# reordenar las columnas del DataFrame df_cartera_de_especies_2

df_cartera_de_especies_2 = df_cartera_de_especies_2.select('cartera','especie','secid','rubro_altair','valor_nominal','valor_contable','fecha_incorporacion_tenencia','fecha_vencimiento','tipo_valoracion','isin')


In [39]:
df_cartera_de_especies_2.show(5)

+-------+-------+-----+------------+-------------+--------------+----------------------------+-----------------+---------------+----+
|cartera|especie|secid|rubro_altair|valor_nominal|valor_contable|fecha_incorporacion_tenencia|fecha_vencimiento|tipo_valoracion|isin|
+-------+-------+-----+------------+-------------+--------------+----------------------------+-----------------+---------------+----+
|    TRD| ARN38D| null|     1250405|         8118|     4241655.0|                  2020-09-08|       2038-01-09|              1|null|
|    TRD| ARN41E| null|     1250405|       151942|   58289663.51|                  2020-09-08|       2041-07-09|              2|null|
|    TRD| AR30DR| null|     1250405|        77966|   45464302.83|                  2020-09-08|       2030-07-09|              2|null|
|    TRD| AR30D5| null|     1250405|         2191|    1160488.27|                  2020-09-08|       2029-07-09|              2|null|
|    TRD|  AS30D| null|     1455453|        50000|    27434250

In [40]:
df_cartera_de_especies_2.write.csv("./santander/cartera_de_especies-20240327.csv", sep=";", header=True, mode="overwrite")